### Initialization

In [1]:
# For Colab only!

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import torch
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch import nn

In [3]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [4]:
print(torch.__version__)
print(torch.cuda.is_available())

1.4.0
True


### Data Loading
MINST data set

In [5]:
import numpy as np

batch_size=200
learning_rate=0.01
epochs=10

In [6]:
(x, y),(x_test, y_test) = keras.datasets.mnist.load_data()

ds_train = tf.data.Dataset.from_tensor_slices((x,y))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def preprocess(x, y):
  x = (tf.cast(x, tf.float32)/255)-0.1307
  y = tf.cast(y, tf.int32)
#   y = tf.one_hot(y,depth=10)   
  return x, y

ds_train = ds_train.map(preprocess).shuffle(1000).batch(batch_size)
ds_test = ds_test.map(preprocess).shuffle(1000).batch(batch_size)


In [39]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

In [8]:
print(type(ds_test))
image, label = next(iter(ds_test))
print(image.shape, label.shape)

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
(200, 28, 28) (200,)


In [9]:
print(type(train_loader))
[image, label] = next(iter(train_loader))
print(image.shape, label.shape)

<class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([200, 1, 28, 28]) torch.Size([200])


### Visualization with tensorboard and visdom

In [27]:
import datetime, os
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(log_dir)

# in terminal: tensorboard --logdir logs


In [40]:
from visdom import Visdom

# terminal run: python -m visdom.server

In [28]:
class FC_model(keras.Model):
    def __init__(self):
        super().__init__()
    
        self.model = keras.Sequential(
            [layers.Dense(200),
            layers.ReLU(),
            layers.Dense(100),
            layers.ReLU(),
            layers.Dense(10)]
            )
    
    def call(self,x):
        x = self.model(x)
        
        return x
    
model = FC_model()
optimizer = tf.optimizers.Adam(learning_rate)
    
globle_step = 0;

for epoch in range(epochs):
    
    for step, (x, y) in enumerate(ds_train):
        x = tf.reshape(x, [-1, 28*28])
        with tf.GradientTape() as tape:            
            logits = model(x)
            
            losses = tf.losses.sparse_categorical_crossentropy(y,logits,from_logits=True)
            loss = tf.reduce_mean(losses)
            
        grads = tape.gradient(loss, model.variables)
        
        optimizer.apply_gradients(zip(grads, model.variables))
        
        if(step%100==0):
            print("epoch:{}, step:{} loss:{}".
                  format(epoch, step, loss.numpy()))
            
            
#             test accuracy: 
            total_correct = 0
            total_num = 0
            
            for x_test, y_test in ds_test:
                x_test = tf.reshape(x_test, [-1, 28*28])
                logits_test = model(x_test)
                
                loss_test = tf.losses.sparse_categorical_crossentropy(y_test, logits_test, from_logits=True)
                loss_test = tf.reduce_mean(loss_test)
                
                y_pred = tf.argmax(logits_test,axis=1)
                
                with summary_writer.as_default():
                    tf.summary.scalar("train_loss", loss.numpy(),step=globle_step)
                    tf.summary.scalar("test_loss", loss_test.numpy(),step=globle_step)
                
                y_pred = tf.cast(y_pred, tf.int32)
                correct = tf.cast((y_pred == y_test), tf.int32)
                correct = tf.reduce_sum(correct)
                
                total_correct += int(correct)
                total_num += x_test.shape[0]
        
            
            accuracy = total_correct/total_num
            print('accuracy: ', accuracy)
            
            with summary_writer.as_default():
                tf.summary.scalar("accuracy", accuracy, step=globle_step)
        
        with summary_writer.as_default():            
            tf.summary.scalar("train_loss_each_step",loss.numpy(),step=globle_step)
        
        globle_step += 1


epoch:0, step:0 loss:2.3398690223693848
accuracy:  0.3156
epoch:0, step:100 loss:0.20715156197547913
accuracy:  0.934
epoch:0, step:200 loss:0.20527370274066925
accuracy:  0.9562
epoch:1, step:0 loss:0.19124512374401093
accuracy:  0.9542
epoch:1, step:100 loss:0.20219218730926514
accuracy:  0.9562
epoch:1, step:200 loss:0.06331567466259003
accuracy:  0.965
epoch:2, step:0 loss:0.0726526603102684
accuracy:  0.957
epoch:2, step:100 loss:0.05037660524249077
accuracy:  0.9691
epoch:2, step:200 loss:0.036688972264528275
accuracy:  0.965
epoch:3, step:0 loss:0.11879907548427582
accuracy:  0.9618
epoch:3, step:100 loss:0.061393316835165024
accuracy:  0.9698
epoch:3, step:200 loss:0.04468255862593651
accuracy:  0.9714
epoch:4, step:0 loss:0.19252124428749084
accuracy:  0.9686
epoch:4, step:100 loss:0.03427652642130852
accuracy:  0.9644
epoch:4, step:200 loss:0.03908463567495346
accuracy:  0.9675
epoch:5, step:0 loss:0.11102376133203506
accuracy:  0.9614
epoch:5, step:100 loss:0.043038588017225

In [ ]:
class FC_NN(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.model = nn.Sequential(
            nn.Linear(28*28, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100,10)
            )
    
    def forward(self, x):
        x = self.model(x)
        
        return x
device = torch.device('cuda:0')

network = FC_NN().to(device)        
optimizer = torch.optim.Adam(network.parameters(),
                            lr=learning_rate)
criteon = torch.nn.CrossEntropyLoss().to(device)

vis = Visdom()

globle_step = 0

for epoch in range(epochs):
    
    for step, (x, y) in enumerate(train_loader):
        x = x.reshape(-1,28*28)
        
        x, y = x.to(device), y.to(device)
        
        logits = network(x)
        loss = criteon(logits, y)
        
        
        vis.line([loss.item()],[globle_step],win='loss_each_step',
                 update='append',opts=dict(title='train_loss_each_step') )
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(step%100 == 0):
            print("epoch:{}, step:{}, loss:{}".
                  format(epoch, step, loss.item()))
        
#             test accuracy
            total_correct = 0
            total_num = 0    

            for x_test, y_test in test_loader:
                    x_test = x_test.reshape(-1,28*28)
                    x_test, y_test = x_test.to(device), y_test.to(device)

                    logits_test = network(x_test)
                    loss_test = criteon(logits_test, y_test)
                    
                    
                    vis.line([[loss.item(), loss_test.item()]],[globle_step], 
                             win='losses', update='append', 
                             opts=dict(title='losses',legend=['train_loss', 'test_loss'] ))
                    
                    y_pred = torch.argmax(logits_test, dim = 1)
                    correct = y_pred == y_test
                    correct = correct.sum()

                    total_correct += correct
                    total_num += x_test.shape[0]

            acc = total_correct.float()/total_num
            print("accuracy: ", acc.item())
            
            vis.line([acc.item()],[globle_step],
                     win='acc', update='append',
                    opts=dict(title = 'accuracy'))
            
            vis.images(x_test.reshape(-1,1,28,28), win='images')
            vis.text(str(y_pred.detach().cpu().numpy()), win = 'pred')
    
        globle_step += 1           
                

Setting up a new session...


epoch:0, step:0, loss:2.312376022338867
accuracy:  0.20260000228881836
epoch:0, step:100, loss:0.24057549238204956
accuracy:  0.9429000020027161
epoch:0, step:200, loss:0.17944090068340302
accuracy:  0.9506999850273132
epoch:1, step:0, loss:0.04460395500063896
accuracy:  0.9589999914169312
epoch:1, step:100, loss:0.10701652616262436
accuracy:  0.9623000025749207
epoch:1, step:200, loss:0.10582515597343445
accuracy:  0.9624999761581421
epoch:2, step:0, loss:0.13842931389808655
accuracy:  0.9690999984741211
epoch:2, step:100, loss:0.13088180124759674
accuracy:  0.9680999517440796
epoch:2, step:200, loss:0.10829789191484451
accuracy:  0.9501000046730042
epoch:3, step:0, loss:0.13796570897102356
accuracy:  0.9673999547958374
epoch:3, step:100, loss:0.08032510429620743
accuracy:  0.9721999764442444
epoch:3, step:200, loss:0.030323320999741554
accuracy:  0.9740999937057495
epoch:4, step:0, loss:0.05643593892455101
accuracy:  0.9724999666213989
epoch:4, step:100, loss:0.06288091838359833
accu